In [12]:
import numpy as np
from scipy.signal import find_peaks
from astropy import constants, units
import numpy as np
import sys
from astropy.io import fits
import matplotlib.pyplot as plt
import sklearn as skl
import pandas as pd
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter as svg
from astropy.constants import c
from astropy.timeseries import LombScargle
from astropy import constants, units
from scipy.signal import find_peaks
from scipy.signal import fftconvolve
import warnings
import glob, os
from astropy.stats import sigma_clip

In [13]:
def read_HERMES(infile):
    #print("%s: Input file is a HERMES file." % infile)
    header = fits.getheader(infile)

    #bjd = header['MJD-OBS']
    # for files with standard wavelegth array
    if ((header['CTYPE1'] == 'WAVELENGTH') or (header['CTYPE1'] == 'AWAV')):
        flux = fits.getdata(infile, byteorder='little')
        crval = header['CRVAL1']
        cdelt = header['CDELT1']
        naxis1 = header['NAXIS1']
        wave = crval + np.arange(0, naxis1) * cdelt

    # for files that are given in logarithmic wl array
    if (header['CTYPE1'] == 'log(wavelength)'):
        flux = fits.getdata(infile, byteorder='little')
        crval = header['CRVAL1']
        cdelt = header['CDELT1']
        naxis1 = header['NAXIS1']
        wave = np.exp(crval + np.arange(0, naxis1)*cdelt)
    else:
        print("Could not read in HERMES fits file - unknown file type.")
        sys.exit()
    return wave, flux

In [14]:
'''
Using numpy fourier transform
'''

def vsini(wave, flux, epsilon=0.6, clam=None, window=None):
    cc = constants.c.to(units.AA / units.s).value

    if window is not None:
        keep = (window[0] <= wave) & (wave <= window[1])
        wave, flux = wave[keep], flux[keep]

    clam = clam or np.mean(wave)

    q1 = 0.610 + 0.062 * epsilon + 0.027 * epsilon ** 2 + 0.012 * epsilon ** 3 + 0.004 * epsilon ** 4

    freqs, ampls = np.fft.fftfreq(len(wave), 0.05), np.abs(np.fft.fft(1 - flux))
    ampls /= max(ampls)

    peaks, _ = find_peaks(-ampls)
    minima = freqs[peaks][:-1]
    minvals = ampls[peaks][:-1]

    freqs = freqs * clam / q1 / cc
    vsini_values = cc / clam * q1 / minima

    error = np.ptp(wave) * clam / q1 / cc

    return (freqs, ampls), (minima, minvals), vsini_values/10**13, error

In [15]:
'''
using astropy oversampled lomb scargle
'''

def vsini(wave, flux, epsilon=0.0, clam=None, window=None):
    cc = constants.c.to(units.AA / units.s).value

    if window is not None:
        keep = (window[0] <= wave) & (wave <= window[1])
        wave, flux = wave[keep], flux[keep]

    clam = clam or np.mean(wave)

    q1 = 0.610 + 0.062 * epsilon + 0.027 * epsilon ** 2 + 0.012 * epsilon ** 3 + 0.004 * epsilon ** 4

    # Using Lomb-Scargle periodogram instead of FFT
    frequency, power = LombScargle(wave, flux).autopower(samples_per_peak=50)

    power /= max(power)  # Normalize power for comparison with ampls

    # Find minima in the power spectrum
    peaks, _ = find_peaks(-power)
    minima = frequency[peaks][:-1]
    minvals = power[peaks][:-1]

    frequency = frequency * clam / q1 / cc
    vsini_values = cc / clam * q1 / minima

    # Estimate the error based on the wavelength range and scaling factor
    error = np.ptp(wave) * clam / q1 / cc

    return (frequency, power), (minima, minvals), vsini_values / 10**13, error

In [ ]:
clam = 6678
wave, flux= read_HERMES('/home/c4011027/PhD_stuff/Other/00943975_HRF_OBJ_ext_CosmicsRemoved_log_merged_cf_norm.fits')
result = vsini(wave, flux, clam = clam, window=[6670, 6686])
print('V(FT) is', max(result[2]), 'km/s')

In [ ]:

epsilon = 0.6
cc = constants.c.to(units.AA / units.s).value
q1 = 0.610 + 0.062 * epsilon + 0.027 * epsilon ** 2 + 0.012 * epsilon ** 3 + 0.004 * epsilon ** 4
freqs = result[0][0] /( clam / q1 / cc)
ampls = result[0][1]
def freq_to_velocity(freqs, clam, q1):
    cc = constants.c.to(units.AA / units.s).value
    return (cc / clam * q1 / freqs)/10**13

# Convert frequencies to velocities
velocities = freq_to_velocity(freqs, clam, q1)
log_ampls = np.log(ampls)


#svgol_freq = svg(log_ampls, window_length= 10, polyorder = 3, mode = 'interp')


# Plot velocity vs amplitude
plt.figure(figsize=(8, 6))
plt.plot(velocities, log_ampls, color = 'darkorange', linewidth = 2)
#plt.plot(velocities, svgol_freq, color = 'red')
plt.ylabel('Amplitude')
plt.xlabel('Velocity (km/s)')
plt.title('Velocity vs Amplitude')
plt.grid(True)
plt.xlim(0,500)
plt.axvline(max(result[2]))
plt.show()

In [25]:
def plot_ft(wave, flux, result, epsilon=0.6, clam=6678, window=[6670, 6686], save_path = 'plots/plot.png'):
    
    cc = constants.c.to(units.AA / units.s).value
    q1 = 0.610 + 0.062 * epsilon + 0.027 * epsilon ** 2 + 0.012 * epsilon ** 3 + 0.004 * epsilon ** 4
    freqs = result[0][0] /( clam / q1 / cc)
    ampls = result[0][1]
    velocities = (cc / clam * q1 / freqs) / 10**13
    log_ampls = np.log(ampls)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 5), sharex=False)
    ax1.plot(velocities, log_ampls, color='darkorange', linewidth=2, label='FT Amplitude')
    ax1.axvline(max(result[2]), color='blue', linestyle='--', label=f'vsini: {max(result[2]):.2f} km/s')
    ax1.set_ylabel('Log Amplitude')
    ax1.set_xlabel('Velocity (km/s)')
    ax1.set_title('Fourier Transform: Velocity vs Log Amplitude')
    ax1.grid(True)
    ax1.set_xlim(0, 500)
    ax1.legend()
    
    buffer_region = [window[0] - 10, window[1] + 10]
    buffer_keep = (buffer_region[0] <= wave) & (wave <= buffer_region[1])
    window_keep = (window[0] <= wave) & (wave <= window[1])
    ax2.plot(wave[buffer_keep], flux[buffer_keep], color='grey', linewidth=1.5, label='Spectrum with Buffer')
    ax2.plot(wave[window_keep], flux[window_keep], color='darkorange', linewidth=1.5, label='Windowed Spectrum')
    ax2.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()

In [19]:
result = vsini(wave, flux, clam=5875, window=[5868, 5882])

In [ ]:
plot_ft(wave, flux, result, epsilon=0.6, clam=5875, window=[5868, 5882])

### FT for random 100 epochs

In [28]:
def read_spectrum(infile):
    data = fits.getdata(infile)
    wave = data[0]
    flux = data[1]
    return wave, flux

def sort_spectra(spectra_files):
    spectra_with_dates = []
    for spectrum_file in spectra_files:
        obs_date = fits.getheader(spectrum_file)['MJD-OBS']
        if obs_date:
            spectra_with_dates.append((spectrum_file, obs_date))
    spectra_with_dates.sort(key=lambda x: x[1])
    sorted_spectra_files = [s[0] for s in spectra_with_dates]
    sorted_obs_dates = [s[1] for s in spectra_with_dates]
    return sorted_spectra_files, sorted_obs_dates

def read_fourier_list(filename):
    line_list = []
    with open(filename, "r") as f:
        for line in f:
            parts = line.strip().split()
            clam = float(parts[0])  # Convert central wavelength to float
            
            # Manually parse the window values without eval
            window_str = parts[1].strip("[]")  # Remove the brackets
            window = [float(val) for val in window_str.split(',')]  # Convert to list of floats
            
            line_list.append({'clam': clam, 'window': window})
    return line_list

def save_result(result, filename):
    frequency, power = result[0]
    minima, minvals = result[1]
    vsini_values = result[2]
    max_length = max(len(frequency), len(power), len(minima), len(minvals), len(vsini_values))

    def pad_to_max_length(arr, length):
        return np.pad(arr, (0, length - len(arr)), constant_values=np.nan)

    frequency = pad_to_max_length(frequency, max_length)
    power = pad_to_max_length(power, max_length)
    minima = pad_to_max_length(minima, max_length)
    minvals = pad_to_max_length(minvals, max_length)
    vsini_values = pad_to_max_length(vsini_values, max_length)
    data_to_save = np.column_stack([frequency, power, minima, minvals, vsini_values])
    header = "Frequency\tPower\tMinima\tMinvals\tvsini_values (km/s)"
    np.savetxt(filename, data_to_save, fmt="%.6f", header=header, delimiter="\t")


def telluric_correction(wv, flx):
    window_size = 100
    cleaned_flx = np.copy(flx)

    for i in range(0, len(wv), window_size):
        flx_window = flx[i:i + window_size]
        clipped_flux = sigma_clip(flx_window, sigma=2, maxiters=10, masked=True)
        cleaned_flx[i:i + window_size] = np.where(clipped_flux.mask, np.nan, flx_window)

    cleaned_flx = np.interp(wv, wv[~np.isnan(cleaned_flx)], cleaned_flx[~np.isnan(cleaned_flx)])
    return wv, cleaned_flx

def process_spectra(line_list, epsilon=0.6):
    results_dir = "/home/c4011027/PhD_stuff/ESO_proposals/checks/ftresults/"
    plots_dir = "/home/c4011027/PhD_stuff/ESO_proposals/checks/ftplots/"
    os.makedirs(results_dir, exist_ok=True)
    os.makedirs(plots_dir, exist_ok=True)
    specfiles, sorted_dates = sort_spectra(glob.glob('/home/c4011027/PhD_stuff/ESO_proposals/manual_normalization/norm/*.fits')[0:100])    
    for line in line_list:
        clam, window = line['clam'], line['window']
        for files in specfiles:
            wv, flx = read_spectrum(files)
            wave, flux = telluric_correction(wv, flx)
            result = vsini(wave, flux, clam=clam, window=window)
            result_filename =results_dir + files.split('/')[-1].replace('.fits', '.txt')
            save_result( result, result_filename)
            plot_filename = plots_dir  + files.split('/')[-1].replace('.fits', '.png')
            plot_ft(wave, flux, result, epsilon=epsilon, clam=clam, window=window, save_path= plot_filename)

In [ ]:
linelist = read_fourier_list('/home/c4011027/PhD_stuff/ESO_proposals/prologs/fourier_linelist.txt')
process_spectra(linelist)

### adding a vsini~v(FT) model to plot

In [15]:
class Model_broad:
    def __init__(self, wave, flux):
        self.x = wave
        self.y = flux

def gauss(x,a,center,R, gamma):
  sigma = sigma = 4471/ (2.0 * R * np.sqrt(2.0 * np.log(2))) 
  return a*np.exp(-(x-center)**2/(2*sigma**2)) + gamma

def Broaden(model, vsini, epsilon=0.5, linear=False, findcont=False):
    # Remove NaN values from the flux array and corresponding wavelength values
    non_nan_idx = ~np.isnan(model.y)
    wvl = model.x[non_nan_idx]
    flx = model.y[non_nan_idx]
    
    dwl = wvl[1] - wvl[0]
    binnu = int(np.floor((((vsini/10)/ 299792.458) * max(wvl)) / dwl)) + 1 #adding extra bins for error handling
    #validIndices = np.arange(len(flx)) + binnu => this was used in rotbroad as a user cond ==> this is always on here
    front_fl = np.ones(binnu) * flx[0]
    end_fl = np.ones(binnu) * flx[-1]
    flux = np.concatenate((front_fl, flx, end_fl))

    front_wv = (wvl[0] - (np.arange(binnu) + 1) * dwl)[::-1]
    end_wv = wvl[-1] + (np.arange(binnu) + 1) * dwl
    wave = np.concatenate((front_wv, wvl, end_wv))

    if not linear:
        x = np.logspace(np.log10(wave[0]), np.log10(wave[-1]), len(wave))
    else:
        x = wave
        
    if findcont:
        # Find the continuum
        model.cont = np.ones_like(flux)  # Placeholder for continuum finding
        
    # Make the broadening kernel
    dx = np.log(x[1] / x[0])
    c = 299792458  # Speed of light in m/s
    lim = vsini / c
    if lim < dx:
        warnings.warn("vsini too small ({}). Not broadening!".format(vsini))
        return Model_broad(wave.copy(), flux.copy())  # Create a copy of the Model object
    
    d_logx = np.arange(0.0, lim, dx)
    d_logx = np.concatenate((-d_logx[::-1][:-1], d_logx))
    alpha = 1.0 - (d_logx / lim) ** 2
    B = (1.0 - epsilon) * np.sqrt(alpha) + epsilon * np.pi * alpha / 4.0  # Broadening kernel
    B /= np.sum(B)  # Normalize

    # Do the convolution
    broadened = Model_broad(wave.copy(), flux.copy())  # Create a copy of the Model object
    broadened.y = fftconvolve(flux, B, mode='same')
    
    return broadened

In [72]:

def generate_broaden(wave, flux, a, gamma, center = 4471, vsini = 350000, epsilon = 0.6, window = [4000, 4100], R = 80000):
    mask = (wave> window[0]) & (wave < window[1])
    wave_m = wave[mask]
    flux_m = flux[mask]    
    instrum = gauss(wave_m, a, center, R, gamma)
    model_broad = Model_broad(wave_m, instrum)
    rotbroad = Broaden(model_broad, vsini)
    return rotbroad.x, rotbroad.y


In [79]:
rotwave , rotflux = generate_broaden(wave, flux, -10, 1, vsini = 280000,  center=4471.48, window=[4460, 4478])
plt.plot(rotwave, rotflux)
plt.xlim(4455, 4488)
plt.ylim(0.8, 1.05)